In [2]:
pwd

'/Users/dblyon/modules/cpr/agotool'

In [3]:
cd app/python

/Users/dblyon/modules/cpr/agotool/app/python


In [4]:
import variables

In [5]:
import igraph as ig
# import plotly.plotly as py
from plotly.graph_objs import *

import os, sys, json
import datetime
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 300) # in order to prevent 50 character cutoff of to_html export / ellipsis
# from plotly_tools import data_bars
import dash
from dash.dependencies import Input, Output, State
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_daq as daq
# import plotly.express as px
import plotly.graph_objects as go

In [107]:
from itertools import combinations
from collections import defaultdict

# ToDo: term_2_connectedTerms_dict: key=term, val=set of terms --> Nodes
def get_term_2_edges_dict(df, lineage_dict_direct):
    """
    iterate over direct parents and stop once at least one is found
    Dict: key=term val=[X_vals_list, Y_vals_list, Weights_list]
    # for every term:
    #   for every edge:
    #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
    #     X_point_list += [Ax, Bx, None]
    #     Y_point_list += [Ay, By, None]
    #     Weights_list += [A_2_B]
    #     X_point_list += [Ax, Cx, None]
    #     Y_point_list += [Ay, Cy, None]    
    """
    term_2_edges_dict = defaultdict(lambda: {"X_points": [], "Y_points": [], "Weights": [], "Nodes": []})
    for etype, group in df.groupby("etype"):
        if etype not in variables.entity_types_with_ontology:
            continue
        funcNameFamilySet_list_merged, funcNameFamilySet_list_temp, edgesXYCoords_list, funcNamePair_set = [], [], [], set()
        funcs_2_travers_set = group["term"]
        for term in funcs_2_travers_set:
            funcNamePair_set |= find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set)
                
        ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
        for funcNamePair in funcNamePair_set:
            X_points_list, Y_points_list, Weight_temp = get_edge_positions_and_weight_as_list_v3(group[group["term"].isin(funcNamePair)])
            funcName_a, funcName_b = funcNamePair
            term_2_edges_dict[funcName_a]["X_points"] += X_points_list
            term_2_edges_dict[funcName_a]["Y_points"] += Y_points_list
            term_2_edges_dict[funcName_a]["Weights"].append(Weight_temp)
            term_2_edges_dict[funcName_a]["Nodes"].append(funcName_b)
            
            term_2_edges_dict[funcName_b]["X_points"] += X_points_list
            term_2_edges_dict[funcName_b]["Y_points"] += Y_points_list
            term_2_edges_dict[funcName_b]["Weights"].append(Weight_temp)
            term_2_edges_dict[funcName_b]["Nodes"].append(funcName_a)
    return term_2_edges_dict     

def find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set):
    funcNamePair_set = set()
    for parents in yield_direct_parents([term], lineage_dict_direct): 
        if not parents:
            continue
        else:
            lineage_2_add = set(parents)
            lineage_2_add.add(term)
            funcNameFamilySet = lineage_2_add.intersection(funcs_2_travers_set)
        len_funcNameFamilySet = len(funcNameFamilySet)
        if len_funcNameFamilySet < 2:
            continue
        elif len_funcNameFamilySet == 2:
            funcNamePair_set.add(tuple(sorted(funcNameFamilySet)))
            return funcNamePair_set
        elif len_funcNameFamilySet > 2:
            for func in funcNameFamilySet - set([term]):
                funcNamePair_set.add(tuple(sorted([term, func])))
            return funcNamePair_set
        else:
            raise StopIteration
    return funcNamePair_set

def get_edge_positions_and_weight_as_list_v3(df):
    """
    # for every term:
    #   for every edge:
    #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
    #     X_point_list += [Ax, Bx, None]
    #     Y_point_list += [Ay, By, None]
    #     Weights_list += [A_2_B]
    #     X_point_list += [Ax, Cx, None]
    #     Y_point_list += [Ay, Cy, None]
    """
    assert df.shape[0] == 2
    
    try:
        first_FG_IDs, second_FG_IDs = df["FG_IDs"].values
        first_FG_IDs = set(first_FG_IDs.split(";"))
        second_FG_IDs = set(first_FG_IDs.split(";"))
        ## compute weight of edge
        jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
    except: #  AttributeError if NaN or ZeroDivisionError if ... 
        jaccard_index = 1
    return df["logFDR"].tolist() + [None], df["effectSize"].tolist() + [None], jaccard_index

def yield_direct_parents(terms, lineage_dict):
    """
    generator: to return all direct parents of given terms
    lineage_dict = cst.get_lineage_dict_for_all_entity_types_with_ontologies(direct_or_allParents="direct")
    terms = ["GO:0016572"]
    for parents in yield_direct_parents(terms, lineage_dict):
        print(parents)
    :param terms: list of string
    :param lineage_dict: child 2 direct parents dict
    :return: list of string
    """
    def get_direct_parents(terms, lineage_dict):
        parents = []
        for term in terms:
            try:
                parents += lineage_dict[term]
            except KeyError:
                pass
        parents = sorted(set(parents))
        return parents
    parents = get_direct_parents(terms, lineage_dict)
    while parents:
        yield parents
        parents = get_direct_parents(parents, lineage_dict)
    return None

# def find_edges_via_hierarchy_v3(df, lineage_dict_direct):
#     """
#     for each etype find edges to draw in plot of x="logFDR", y="effectSize"
#     add funcEnum and lineage to set
#     v1: merge sets if there is overlapp --> wrong since everything gets connected
#     v2: don't merge set, simply check if >= 2 funcEnums from given user data present
#     filter by funcEnums that are in the given user data
#     v3: iterate over direct parents and stop once at least one is found
#     e.g. of return
#     {-57: [],
#      -51: [{47939, 47977, 48117, 48335, 48487}],
#      ...}     
#     df: DataFrame of aGOtool output with ['funcEnum', 'etype'] cols
#     lineage_dict_enum: key: function enumeration, value: set of func enum array all parents
#     :return: etype_2_funcEnumFamilySet_list_dict, key: etype(int), val: list of sets (int)
#     """
#     x_startL, x_stopL, y_startL, y_stopL, weightL, etypeL = [], [], [], [], [], []
#     etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = {}, {} 
#     for etype, group in df.groupby("etype"):
#         funcNameFamilySet_list_merged, funcNameFamilySet_list_temp, edgesXYCoords_list, funcNamePair_set = [], [], [], set()
#         if etype not in variables.entity_types_with_ontology:
#             continue
#         funcs_2_travers_set = group["term"]
#         for term in funcs_2_travers_set:
#             funcNamePair_set |= find_pairs_2_link(term, lineage_dict_direct, funcs_2_travers_set)
                
#         ## extract x and y coordinates and a weight depending on the overlapp of FG_IDs (Jaccard index)
#         for funcNamePair in funcNamePair_set:
#             t = get_edge_positions_and_weight_as_list(group[group["term"].isin(funcNamePair)])
#             x_startL_2merge, x_stopL_2merge, y_startL_2merge, y_stopL_2merge, weightL_2merge = t
#             x_startL += x_startL_2merge
#             x_stopL += x_stopL_2merge
#             y_startL += y_startL_2merge
#             y_stopL += y_stopL_2merge
#             weightL += weightL_2merge
#             etypeL += len(weightL_2merge) * [etype]                                        
#         etype_2_funcNamePair_set_dict[etype] = funcNamePair_set
#     df_coords = pd.DataFrame()
#     df_coords["etype"] = etypeL
#     df_coords["x_start"] = x_startL
#     df_coords["x_stop"] = x_stopL    
#     df_coords["y_start"] = y_startL
#     df_coords["y_stop"] = y_stopL
#     df_coords["weight"] = weightL        
#     return etype_2_funcNamePair_set_dict, df_coords #etype_2_edgesXYCoords_list_dict
# 
# def get_edge_positions_and_weight_as_list_v2(df):
#     """
#     # for every term:
#     #   for every edge:
#     #     e.g. between point A (Ax, Ay) and B (Bx, By) as well as A (Ax, Ay) and C (Cx, Cy)
#     #     X_point_list += [Ax, Bx, None]
#     #     Y_point_list += [Ay, By, None]
#     #     Weights_list += [A_2_B]
#     #     X_point_list += [Ax, Cx, None]
#     #     Y_point_list += [Ay, Cy, None]
#     """
#     X_point_list, Y_point_list, Weights_list = [], [], []
#     for first_index, second_index in combinations(df.index, 2):
#         try:
#             first_FG_IDs = set(df.loc[first_index, "FG_IDs"].split(";"))
#             second_FG_IDs = set(df.loc[second_index, "FG_IDs"].split(";"))
#             ## compute weight of edge
#             jaccard_index = len(first_FG_IDs.intersection(second_FG_IDs)) / len(first_FG_IDs.union(second_FG_IDs))
#         except: #  AttributeError if NaN or ZeroDivisionError if ... 
#             jaccard_index = 1
#         X_points_list += df.loc[[first_index, second_index], "logFDR"].tolist() + [None]
#         Y_points_list += df.loc[[first_index, second_index], "effectSize"].tolist() + [None]
#         Weights_list.append(jaccard_index)
#     return X_points_list, Y_points_list, Weights_list

In [50]:
import create_SQL_tables_snakemake as cst

In [51]:
lineage_dict = cst.get_lineage_dict_for_all_entity_types_with_ontologies(direct_or_allParents="direct")

In [108]:
term_2_edges_dict = get_term_2_edges_dict(df, lineage_dict)

In [109]:
term_2_edges_dict["KW-0539"]

{'X_points': [], 'Y_points': [], 'Weights': [], 'Nodes': []}

In [93]:
pwd

'/Users/dblyon/modules/cpr/agotool/app/python'

In [101]:
term_2_edges_dict = dict(term_2_edges_dict)

In [102]:
import pickle
pickle.dump(term_2_edges_dict, open("term_2_edges_dict.p", "wb"))

In [86]:
# for term in term_2_edges_dict:
#     for weight in term_2_edges_dict[term]["Weights"]:
#         if weight < 1:
#             print(term, weight)

In [56]:
df = pd.read_csv(variables.fn_example, sep="\t")
etype_2_funcNamePair_set_dict, etype_2_edgesXYCoords_list_dict = find_edges_via_hierarchy_v3(df, lineage_dict)

In [87]:
# df.head()

In [72]:
first_index=0
second_index=4
df.loc[[first_index, second_index], "logFDR"].tolist() + [None]

[4.671470586861855, 4.671470586861855, None]

In [77]:
x = df.loc[[first_index, second_index], ]
x

,term,hierarchical_level,description,year,over_under,p_value,FDR,effectSize,ratio_in_FG,ratio_in_BG,...,FG_IDs,s_value,rank,funcEnum,category,etype,logFDR,FG_count_2_circle_size,rank_2_transparency,color
0,GOCC:0005634,8,Nucleus,-1,o,0.000004,0.009358,0.133736,0.522002,0.388266,...,P31383;P37898;P14164;P40535;P31787;Q08981;P52910;P60010;Q02336;P53909;Q2V2Q1;P47143;Q01976;P07248;Q12433;Q12156;P10869;P47019;P16550;P38281;P32454;P53940;P22936;P40024;Q04371;P08566;P32449;P80428;P53946;Q05123;P32447;P41696;P22035;Q12186;P47176;Q06338;P35817;Q06631;P43583;P29311;P34730;Q08965;P4...,0.716327,1,254,Gene Ontology cellular component TEXTMINING,-20,4.671471,11.543799,1.0,#1b9e77
4,GOCC:0043232,7,Intracellular non-membrane-bounded organelle,-1,o,0.000017,0.009358,0.088870,0.459016,0.370147,...,P31383;Q02486;P15891;P40535;Q08981;P19414;P52910;P60010;Q04894;P38266;Q12156;P10869;P46367;P46672;P11076;P32449;P47117;P80428;P53946;Q05123;Q05933;P32447;P07251;P00830;P22035;P35817;Q06631;P29311;Q08965;P40493;Q07457;P53741;Q04347;P25627;P23293;P06787;P28495;P34237;P17106;P32504;P33322;P21560;P0...,0.424281,5,1879,Gene Ontology cellular component TEXTMINING,-20,4.671471,10.150911,0.6,#1b9e77


In [78]:
x["FG_IDs"]

0    P31383;P37898;P14164;P40535;P31787;Q08981;P52910;P60010;Q02336;P53909;Q2V2Q1;P47143;Q01976;P07248;Q12433;Q12156;P10869;P47019;P16550;P38281;P32454;P53940;P22936;P40024;Q04371;P08566;P32449;P80428;P53946;Q05123;P32447;P41696;P22035;Q12186;P47176;Q06338;P35817;Q06631;P43583;P29311;P34730;Q08965;P4...
4    P31383;Q02486;P15891;P40535;Q08981;P19414;P52910;P60010;Q04894;P38266;Q12156;P10869;P46367;P46672;P11076;P32449;P47117;P80428;P53946;Q05123;Q05933;P32447;P07251;P00830;P22035;P35817;Q06631;P29311;Q08965;P40493;Q07457;P53741;Q04347;P25627;P23293;P06787;P28495;P34237;P17106;P32504;P33322;P21560;P0...
Name: FG_IDs, dtype: object

In [58]:
etype_2_edgesXYCoords_list_dict

,etype,x_start,x_stop,y_start,y_stop,weight
0,-51,6.976356,6.976356,-0.087144,-0.070751,0.435897
1,-51,6.976356,6.976356,-0.071613,-0.070751,0.985507
2,-51,6.976356,6.976356,0.075928,0.073339,0.922581
3,-51,6.976356,6.976356,-0.087144,-0.071613,0.442308
4,-22,6.396531,6.396531,0.124245,0.047455,0.330065
...,...,...,...,...,...,...
94,-20,4.671471,4.671471,0.125108,0.057808,0.309735
95,-20,4.671471,3.641455,0.059534,0.068162,0.172193
96,-20,4.671471,4.671471,0.088007,0.064711,0.341418
97,-20,4.671471,4.671471,0.125108,0.044003,0.306267


In [63]:
for parents in yield_direct_parents(["GO:0000001"], lineage_dict):
    print(parents)

['GO:0048308', 'GO:0048311']
['GO:0006996', 'GO:0007005', 'GO:0051646']
['GO:0006996', 'GO:0016043', 'GO:0051640']
['GO:0016043', 'GO:0051641', 'GO:0071840']
['GO:0009987', 'GO:0051179', 'GO:0071840']
['GO:0008150', 'GO:0009987']
['GO:0008150']


In [65]:
lineage_dict['GO:0048311']

['GO:0007005', 'GO:0051646']